In [6]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
!pip install chromadb sentence-transformers python-multipart tqdm pandas numpy
!pip install --upgrade sentence-transformers

import chromadb
from chromadb.utils import embedding_functions
import json
import os
from tqdm import tqdm
from google.colab import files
import zipfile
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

# Configuration
MODEL_NAME = "all-mpnet-base-v2"  # Modèle performant pour l'embedding
BATCH_SIZE = 50  # Taille optimale des lots
MIN_KEYWORDS = 5  # Nombre minimum de mots-clés

# Téléchargement des fichiers
uploaded = files.upload()
fichiers_json = list(uploaded.keys())

def preprocess_text(text):
    """Nettoyage et normalisation du texte"""
    if not text:
        return ""
    text = text.replace('\n', ' ').replace('\r', ' ').strip()
    return ' '.join(text.split())[:10000]  # Limite à 10k caractères

def extract_keywords(item):
    """Extraction des mots-clés"""
    keywords = item.get('keywords', [])
    if isinstance(keywords, list):
        return list(set(keywords))[:10]  # 10 mots-clés max
    return []

def prepare_course_content(course):
    """Préparation du contenu des cours"""
    if not isinstance(course.get('content'), list):
        return ""

    sections = []
    for section in course['content']:
        if isinstance(section, dict):
            section_text = f"{section.get('element', '')} {section.get('section', '')}"
            sections.append(preprocess_text(section_text))
    return " ".join(sections)

def charger_donnees(fichiers):
    """Charge et prépare les données"""
    documents = []
    metadatas = []
    ids = []
    failed_items = 0

    for fichier in fichiers:
        with open(fichier, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                if not isinstance(data, list):
                    data = [data]
            except json.JSONDecodeError:
                print(f"Erreur de lecture du fichier {fichier}")
                continue

        for item in tqdm(data, desc=f"Traitement de {fichier}"):
            try:
                # Préparation du contenu selon le type
                if item["type"] == "article":
                    texte_complet = f"{item['titre']}\n{item['description']}\n{item['contenu']}"
                elif item["type"] == "video":
                    texte_complet = f"{item['titre']}\n{item['description']}\n{item.get('transcription', '')}"
                elif item["type"] == "course":
                    texte_complet = f"{item['title']}\n{item['description']}\n{prepare_course_content(item)}"
                else:
                    continue

                texte_complet = preprocess_text(texte_complet)
                if not texte_complet:
                    continue

                # Métadonnées de base
                metadata = {
                    "type": item["type"],
                    "domaine": item.get("domaine", item.get("domain", "inconnu")),
                    "titre": item.get("titre", item.get("title", "Sans titre")),
                    "url": item.get("url", ""),
                    "source": item.get("source", item.get("chaine", item.get("author", "Inconnu"))),
                    "date": item.get("date_pub", ""),
                    "keywords": "|".join(extract_keywords(item))
                }

                # Métadonnées spécifiques
                if item["type"] == "course":
                    metadata["author"] = item.get("author", "")
                    metadata["level"] = item.get("level", "beginner")
                elif item["type"] == "video":
                    metadata["duration"] = item.get("durée", item.get("duration", ""))

                documents.append(texte_complet)
                metadatas.append(metadata)
                # ID unique combinant plusieurs champs
                ids.append(f"{item['id']}_{item['type']}_{item.get('domaine', 'unknown')}")

            except Exception as e:
                failed_items += 1
                continue

    print(f"\nTerminé avec {failed_items} éléments ignorés")
    return documents, metadatas, ids

# Chargement des données
documents, metadatas, ids = charger_donnees(fichiers_json)

# Initialisation de ChromaDB
client = chromadb.PersistentClient(path="/content/chroma_db_improved")

# Modèle d'embedding
model = SentenceTransformer(MODEL_NAME)
print(f"\nModèle d'embedding chargé: {MODEL_NAME}")

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=MODEL_NAME
)

# Création de la collection avec paramètres optimisés
collection = client.get_or_create_collection(
    name="stem_contents_v2",
    embedding_function=sentence_transformer_ef,
    metadata={
        "hnsw:space": "cosine",
        "hnsw:M": 16,
        "hnsw:construction_ef": 200,
        "hnsw:search_ef": 100
    }
)

# Filtrage des documents valides
valid_indices = [i for i, doc in enumerate(documents) if doc and len(doc) > 20]
filtered_docs = [documents[i] for i in valid_indices]
filtered_metas = [metadatas[i] for i in valid_indices]
filtered_ids = [ids[i] for i in valid_indices]

print(f"\nDocuments valides pour l'indexation: {len(filtered_docs)}/{len(documents)}")

# Indexation par lots
for i in tqdm(range(0, len(filtered_docs), BATCH_SIZE), desc="Indexation"):
    batch_docs = filtered_docs[i:i+BATCH_SIZE]
    batch_metas = filtered_metas[i:i+BATCH_SIZE]
    batch_ids = filtered_ids[i:i+BATCH_SIZE]

    try:
        collection.add(
            documents=batch_docs,
            metadatas=batch_metas,
            ids=batch_ids
        )
    except Exception as e:
        print(f"\nErreur lors de l'ajout du lot {i//BATCH_SIZE}: {str(e)}")
        continue

print(f"\nIndexation terminée. {collection.count()} documents indexés.")

def improved_search(query, domain=None, content_type=None, n_results=5):
    """Recherche améliorée avec scoring personnalisé"""
    filters = {}
    if domain:
        filters["domaine"] = domain
    if content_type:
        filters["type"] = content_type

    # Recherche initiale large
    results = collection.query(
        query_texts=[query],
        n_results=min(n_results*3, 50),
        where=filters if filters else None
    )

    if not results['ids'] or not results['ids'][0]:
        return {"ids": [], "metadatas": [], "documents": []}

    # Création du DataFrame pour le tri
    df = pd.DataFrame({
        'id': results['ids'][0],
        'metadata': results['metadatas'][0],
        'document': results['documents'][0],
        'distance': results['distances'][0]
    })

    # Calcul du score de pertinence
    def relevance_score(row):
        score = 1 - (row['distance'] / 2)  # Similarité cosine normalisée

        # Bonus pour les mots-clés correspondants
        keywords = row['metadata'].get('keywords', '').lower().split('|')
        query_terms = query.lower().split()
        keyword_matches = sum(1 for term in query_terms if any(term in kw for kw in keywords))
        score += 0.15 * min(keyword_matches, 5)  # Poids augmenté

        # Bonus pour le titre
        title = row['metadata'].get('titre', '').lower()
        title_matches = sum(1 for term in query_terms if term in title)
        score += 0.1 * title_matches  # Poids augmenté

        return score

    df['relevance'] = df.apply(relevance_score, axis=1)
    df = df.sort_values('relevance', ascending=False).head(n_results)

    return {
        'ids': df['id'].tolist(),
        'metadatas': df['metadata'].tolist(),
        'documents': df['document'].tolist(),
        'distances': (1 - df['relevance']).tolist()
    }

# Exemple de requête
query = "conception de train innovante"
print(f"\nRecherche pour: '{query}'")
results = improved_search(query, n_results=3)

# Affichage des résultats
print("\nMeilleurs résultats:")
for i, (id, meta) in enumerate(zip(results['ids'], results['metadatas'])):
    print(f"\n{i+1}. {meta['titre']} ({meta['type']})")
    print(f"Domaine: {meta['domaine']}")
    print(f"Source: {meta['source']}")
    print(f"URL: {meta['url']}")
    print(f"Mots-clés: {meta.get('keywords', '')[:100]}...")

# Sauvegarde de la base de données
def save_database():
    zip_path = 'chroma_db_improved.zip'
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        if os.path.exists('/content/chroma_db_improved'):
            for root, dirs, files in os.walk('/content/chroma_db_improved'):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, '/content/chroma_db_improved')
                    zipf.write(file_path, arcname)
            print("\nCompression terminée.")
        else:
            print("Erreur: Dossier non trouvé")
            return

    if os.path.exists(zip_path):
        from google.colab import files
        files.download(zip_path)
    else:
        print("Erreur création ZIP")

save_database()

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.0.1%2Bcu118-cp311-cp311-linux_x86_64.whl (2267.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89990 sha256=40f3dd7c89cfdbebd326cbcdf503758fe3c9ff86c33e6da0e72e17c34431fff1
  Stored in directory: /root/.cache/pip/wheels/fc/5d/45/34fe9945d5e45e261134e72284395be36c2d4828af38e2b0fe
Successfully built lit
  Attempting uninstall: triton
    Found existing installation: triton 3.3.0
    Uninstalling triton-3.3.0:
      Successfully uninstalled triton-3.3.0


Saving articles_engineering.json to articles_engineering (4).json
Saving articles_mathematics.json to articles_mathematics (4).json
Saving articles_science.json to articles_science (4).json
Saving articles_technology.json to articles_technology (4).json
Saving courses_engineering.json to courses_engineering (4).json
Saving courses_mathematics.json to courses_mathematics (4).json
Saving courses_science.json to courses_science (4).json
Saving courses_technology.json to courses_technology (4).json
Saving videos_engineering.json to videos_engineering (4).json
Saving videos_mathematics.json to videos_mathematics (4).json
Saving videos_science.json to videos_science (4).json
Saving videos_technology.json to videos_technology (4).json


Traitement de videos_technology (4).json: 100%|██████████| 25/25 [00:00<00:00, 215.17it/s]



Terminé avec 0 éléments ignorés

Modèle d'embedding chargé: all-mpnet-base-v2

Documents valides pour l'indexation: 150/150


Indexation: 100%|██████████| 3/3 [04:10<00:00, 83.59s/it]



Indexation terminée. 250 documents indexés.

Recherche pour: 'conception de train innovante'

Meilleurs résultats:

1. River-Inspired Interior Gives a Train a Softer and Gentler Character (article)
Domaine: engineering
Source: Yanko Design
URL: https://www.yankodesign.com/2025/03/28/river-inspired-interior-gives-a-train-a-softer-and-gentler-character/
Mots-clés: sleek metal|images of sleek|marvel of engineering|TGV high-speed rail|TGV Inoui|iconic TGV high-spee...

2. River-Inspired Interior Gives a Train a Softer and Gentler Character (article)
Domaine: engineering
Source: Yanko Design
URL: https://www.yankodesign.com/2025/03/28/river-inspired-interior-gives-a-train-a-softer-and-gentler-character/
Mots-clés: sleek metal|images of sleek|marvel of engineering|TGV high-speed rail|TGV Inoui|iconic TGV high-spee...

3. Industrial Design Case Study: Speck Design Tackles a Smart Frying Pan (article)
Domaine: engineering
Source: Core77.com
URL: https://www.core77.com/posts/135980/Industrial-

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>